**Import Libraries and Define constants**

In [44]:
#### Training based on features of audio
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sknn.mlp import Classifier, Layer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import pickle
import numpy as np
import random
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier

#Constant
EMOTION_ANNOTATORS = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}

#EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2}




**Load data**

In [45]:

##Loading  data from files
filehandlerInput = open('processed-data/input.obj', 'rb')
filehandlerOutput = open('processed-data/output.obj', 'rb')
input = pickle.load(filehandlerInput)
output = pickle.load(filehandlerOutput)
print("Size input, output:", len(input),", ", len(output))

Size input, output: 7513 ,  7513


**Analyze data**

In [46]:
feature_name= ['energy', 
               'f0', 'intensity', 'f1', 'f2', 'f3','f1-bw','f2-bw','f3-bw' ,
               'f2-f1', 'f3-f1', 
               'jitter', 'shimmer', 'duration',
              'unvoiced_percent', 'breaks_degree', 'max_dur_pause', 'average_dur_pause']

num_feas = len(input[0])

numNan = [np.count_nonzero(np.isnan(input[:,i]))   for i in range (0, num_feas)]
print("Number of Nan values in each features in all sample:", numNan)

# index_fea_contain_Nan = [i for i in range(0, len(numNan)) if numNan[i] != 0]
# print("Index of features containing Nan values: ", index_fea_contain_Nan)

# fea_contain_Nan = [feature_name[index] for index in index_fea_contain_Nan]
# print("Name of features containing Nan values: ", fea_contain_Nan)

# Filter samples containing Nan values
input_filtered = input[~np.any(np.isnan(input), axis=1)]
output_filtered = output[~np.any(np.isnan(input), axis=1)]
print("Size filteres input, output: ", len(input_filtered), ", ", len(output_filtered))

#Normalize input
input_filtered = (input_filtered - input_filtered.min(axis=0)) / (input_filtered.max(axis=0) - input_filtered.min(axis=0))

print(EMOTION)


Number of Nan values in each features in all sample: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0]
Size filteres input, output:  7506 ,  7506
{'ang': 0, 'hap': 1, 'sad': 2, 'neu': 3, 'fru': 4, 'exc': 5, 'fea': 6, 'sur': 7, 'dis': 8, 'oth': 9, 'xxx': 10}


**Cluster emotion**

In [47]:
# exc -> hap. Positive
output_filtered[output_filtered == 5] = 1

#sad -> ang. Negative
#output_filtered[output_filtered == 2] = 0
#output_filtered[output_filtered == 4] = 0


**Preprocess data**

In [48]:
def printQuantitySample(output):
    y = np.bincount(output)
    ii = np.nonzero(y)[0]
    a = list(zip(ii, y[ii]))
    print("EMOTION_ANNOTATE: ", EMOTION)
    print("\nThe quantity of each label: ", a, "\n")
    
def filterLabels(input, output, labels=['ang','hap','sad']):
    labels_int = [EMOTION[l] for l in labels]
    condition = [out in labels_int for out in output]
    input = input[condition]
    output = output[condition]
    return input, output
    
    
# Remove labels that have small quantity.
input_filtered, output_filtered = filterLabels(input_filtered, output_filtered, ['ang', 'sad', 'hap' ])
printQuantitySample(output_filtered)

#Shuffer
c = list(zip(input_filtered, output_filtered))
random.shuffle(c)
input_filtered, output_filtered = zip( * c)
input_filtered = np.array(input_filtered)
output_filtered = np.array(output_filtered)



EMOTION_ANNOTATE:  {'ang': 0, 'hap': 1, 'sad': 2, 'neu': 3, 'fru': 4, 'exc': 5, 'fea': 6, 'sur': 7, 'dis': 8, 'oth': 9, 'xxx': 10}

The quantity of each label:  [(0, 1102), (1, 1633), (2, 1076)] 



**Split data**

In [49]:
X_train, X_test, y_train, y_test = train_test_split(input_filtered, output_filtered, test_size=0.2, random_state=300)
print("Size training, testing set: ", len(X_train), ", ", len(X_test))
print(y_train)

Size training, testing set:  3048 ,  763
[1 1 1 ... 1 0 2]


**Search best parameters for RandomForest model**

In [7]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 4)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 30, num = 5)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

#Tuninng parameter
# rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_train, y_train)
# print(rf_random.best_params_)
    


{'n_estimators': [200, 466, 733, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 15, 20, 25, 30, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


**Search model playing around**

In [8]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
clf =  MLPClassifier(solver='lbfgs', alpha=1e-7,
                 hidden_layer_sizes=(300), random_state=200)

log_loss = cross_val_score(clf, X_train, y_train, scoring= 'neg_log_loss', cv = 5) 
accs = cross_val_score(clf, X_train, y_train, scoring= 'accuracy', cv = 5) 
print("Log-loss: ", log_loss)
print("Accuracy: ",  accs)



Log-loss:  [-0.63353026 -0.74091926 -0.65012047 -0.65834728 -0.76786972]
Accuracy:  [0.71214953 0.71535581 0.70356473 0.69230769 0.68667917]


**Training**

In [50]:
def training(X_train, y_train):    
    sm = SMOTE(random_state=42)
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
    i_fold = 0
    accuracy_train_results = []
    accuracy_valid_results = []

    for train_index, valid_index in kf.split(X_train):
        i_fold = i_fold + 1
        
        x_train_sub, x_valid_sub = X_train[train_index], X_train[valid_index]
        y_train_sub, y_valid_sub = y_train[train_index], y_train[valid_index]
        
        #clf = RandomForestClassifier(n_estimators = 300)
        clf = MLPClassifier(solver='lbfgs', alpha=1e-7,
                 hidden_layer_sizes=(300), random_state=200)
        
       #print(clf.get_params())
        #Upsampling train data
        x_train_sub, y_train_sub = sm.fit_sample(x_train_sub, y_train_sub)
        clf.fit(x_train_sub, y_train_sub)
        
        score = clf.score(x_train_sub, y_train_sub)
        score1 = clf.score(x_valid_sub, y_valid_sub)
        accuracy_train_results.append(score)
        accuracy_valid_results.append(score1)
        
        print("Score of training set: ", score)
        print("Score of validation set: ", score1)
     
       
    
    avg_accuracy_train_result = np.sum(accuracy_train_results) / len(accuracy_train_results)
    avg_accuracy_valid_result = np.sum(accuracy_valid_results) / len(accuracy_valid_results)
    print("Average accuracy training set, std:", avg_accuracy_train_result, " ",\
          np.std(accuracy_train_results))
    print("Average accuracy validation set, std:", avg_accuracy_valid_result," ", \
          np.std(accuracy_valid_results))     
    
    
    #Upsampling data
    X_train, y_train = sm.fit_sample(X_train, y_train)
    printQuantitySample(y_train)
    clf.fit(X_train, y_train)
    
    return clf
   
# Training 
clf = training(X_train, y_train)

#Save model  into files
filename = 'model/model1.sav'
pickle.dump(clf, open(filename, 'wb'))
print("Saved model into file")

Score of training set:  0.8039381153305204
Score of validation set:  0.7016393442622951
Score of training set:  0.8011299435028248
Score of validation set:  0.7245901639344262
Score of training set:  0.8090395480225989
Score of validation set:  0.6786885245901639
Score of training set:  0.8020477815699659
Score of validation set:  0.6688524590163935
Score of training set:  0.8028248587570621
Score of validation set:  0.6786885245901639
Score of training set:  0.8159577425632472
Score of validation set:  0.7311475409836066
Score of training set:  0.8154960981047937
Score of validation set:  0.7180327868852459
Score of training set:  0.8168067226890756
Score of validation set:  0.6983606557377049
Score of training set:  0.806325216904562
Score of validation set:  0.7105263157894737
Score of training set:  0.8009531819456126
Score of validation set:  0.7203947368421053
Average accuracy training set, std: 0.8074519209390264   0.006111647806578266
Average accuracy validation set, std: 0.703

**Test **

In [51]:
def test(clf, X_test, y_test):
    predicts = clf.predict(X_test)
   # pro = clf.predict_proba(X_test)

    score_test = clf.score(X_test, y_test)
    print("\nScore for test set: ", score_test)
    
    matrix = confusion_matrix(y_test, predicts)
    print ("\nConfusion matrix:..................... \n",matrix)
    
    sum_colum = np.sum(matrix, axis = 0)
    sum_row = np.sum(matrix, axis = 1)

    TP = [matrix[i,i] for i in range(0, len(matrix))]
    print("\nTP: ", TP,"\n")   
    FP = [sum_colum[i] - matrix[i,i] for i in range(0, len(matrix))]
    print("FP: ", FP,"\n")
    FN = [sum_row[i] - matrix[i,i] for i in range(0, len(matrix))]
    print("FN: ", FN,"\n")
    Presision = [TP[i] /(TP[i] + FP[i])  for i in range(0, len(matrix))]
    Recall = [TP[i] /(TP[i] + FN[i])  for i in range(0, len(matrix))]
    F1_score = [2 * Presision[i] * Recall[i] /(Presision[i] + Recall[i])  for i in range(0, len(matrix))]
    
    print("\nPrecision: ", Presision,"\n")
    print("Recall: ", Recall,"\n")
    print("F1_scrore: ", F1_score, "\n")

test(clf, X_test, y_test)




Score for test set:  0.7339449541284404

Confusion matrix:..................... 
 [[153  45   9]
 [ 65 204  46]
 [ 11  27 203]]

TP:  [153, 204, 203] 

FP:  [76, 72, 55] 

FN:  [54, 111, 38] 


Precision:  [0.6681222707423581, 0.7391304347826086, 0.7868217054263565] 

Recall:  [0.7391304347826086, 0.6476190476190476, 0.8423236514522822] 

F1_scrore:  [0.7018348623853211, 0.6903553299492385, 0.8136272545090181] 



**Test probability**

In [53]:
from sklearn.metrics import log_loss

predicts = clf.predict(X_test)
pro = clf.predict_proba(X_test)

theshold_consitent = 0.70
consitent_predictions = pro[[np.any(p> theshold_consitent)  for p in pro]]
print("Ratio of cositent prediction: ", len(consitent_predictions) / len (pro))
print(pro[0:4])
print(y_test[0:4])
print(predicts[0:4])
    
log_loss_value = log_loss(y_test, pro, labels=[0,1,2])
print("Log loss value:", log_loss_value)
print("Prediciton accuracy based on log loss: ", np.exp(-log_loss_value))




Ratio of cositent prediction:  0.7378768020969856
[[9.90556762e-01 9.44282693e-03 4.11136213e-07]
 [7.56307833e-02 6.14036932e-01 3.10332285e-01]
 [2.98540608e-03 1.71411268e-02 9.79873467e-01]
 [9.70173380e-01 2.94957693e-02 3.30850611e-04]]
[1 2 2 0]
[0 1 2 0]
Log loss value: 0.6819651252836908
Prediciton accuracy based on log loss:  0.5056224040701159


**Training on keras**

In [42]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation


y_train_cat = to_categorical(y_train, num_classes=None)
y_test_cat = to_categorical(y_test, num_classes=None)

print(y_train_cat)
model = Sequential([
    Dense(128, input_shape=(88,)),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(48),
    Activation('relu'),
    Dense(24),
    Activation('relu'),
    Dense(2),
    Activation('softmax'),
])
model.compile(optimizer='Adagrad',
              loss='category_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train_cat, validation_data = (X_test, y_test_cat), epochs = 100)





[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]
Train on 2668 samples, validate on 667 samples
Epoch 1/100
2668/2668 [==============================] - 1s 285us/step - loss: 0.6596 - acc: 0.5971 - val_loss: 0.6125 - val_acc: 0.6582
Epoch 2/100
2668/2668 [==============================] - 0s 109us/step - loss: 0.6016 - acc: 0.6724 - val_loss: 0.6131 - val_acc: 0.6507
Epoch 3/100
2668/2668 [==============================] - 0s 109us/step - loss: 0.5873 - acc: 0.6852 - val_loss: 0.5869 - val_acc: 0.6807
Epoch 4/100
2668/2668 [==============================] - 0s 112us/step - loss: 0.5729 - acc: 0.7069 - val_loss: 0.5787 - val_acc: 0.6732
Epoch 5/100
2668/2668 [==============================] - 0s 109us/step - loss: 0.5653 - acc: 0.7076 - val_loss: 0.5774 - val_acc: 0.6837
Epoch 6/100
2668/2668 [==============================] - 0s 113us/step - loss: 0.5593 - acc: 0.7151 - val_loss: 0.6262 - val_acc: 0.6597
Epoch 7/100
2668/2668 [==============================] - 0s 107us/step 

In [40]:
predicts = model.predict_classes(X_test)
#print(predicts)
#print(y_test)
acc = predicts == y_test
print(np.count_nonzero(acc) / len(predicts))

0.7076461769115442
